# Sklearn

## sklearn.grid_search
Подбор параметров по сетке

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

### Генерация датасета

In [2]:
iris = datasets.load_iris()

In [3]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [4]:
classifier = linear_model.SGDClassifier(random_state = 0) 

### Генерация сетки

Возможные параметры классификатора

In [5]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

Параметры

In [6]:
parameters_grid = {
    'loss' : ['hinge', 'log',
              'squared_hinge', 'squared_loss'], #функция потерь
    'penalty' : ['l1', 'l2'], #регуляризаторы
    'max_iter' : np.arange(5,10), #количество итераций
    'alpha' : np.linspace(0.0001, 0.001, num = 5), #множитель регуляризации
}

Стратегия кросс-валидациии

In [7]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10,
                                             test_size = 0.2,
                                             random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

Полный перебор по сетке

In [8]:
grid_cv = model_selection.GridSearchCV(classifier, #классификатора
                                   parameters_grid, #сетка
                                   scoring = 'accuracy', #метрика
                                   cv = cv) #стратегия

Обучение сетки

In [9]:
%%time

grid_cv.fit(train_data, train_labels)

CPU times: user 4.57 s, sys: 15.2 ms, total: 4.59 s
Wall time: 4.62 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                 

Лучший классификатор

In [10]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [11]:
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


Все результаты

In [12]:
grid_cv.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_alpha', 'param_loss', 'param_max_iter', 'param_penalty', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

#### В model_seliction нет grid_score и не понятно, что тут вызвать

#### Randomized grid search

Случайный поиск по сетке - проверяет случайные точки из сетки в поисках лучшей.

In [13]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier,
                                                    parameters_grid,
                                                    scoring = 'accuracy',
                                                    cv = cv,
                                                    n_iter = 20, #количество точек из полной сетки
                                                    random_state = 0)

Работает значительно быстрее

In [14]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 503 ms, sys: 3.93 ms, total: 507 ms
Wall time: 506 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score='raise-deprecating',
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n...
                                           tol=0.001, validation_fraction=0.1,
                                           verbose=0, warm_start=False),
                   iid='warn', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000

In [15]:
print (randomized_grid_cv.best_score_)
print (randomized_grid_cv.best_params_)

0.8666666666666667
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}
